<a href="https://colab.research.google.com/github/imthelizardking/CMP784--Final--Project/blob/main/Keras_LSTM_training_on_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding

In [ ]:
# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words (among top max_features most common words)
maxlen = 500

# Load IMDB sentiment data. Skip top most common words.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features,
                                                      skip_top=10)

# Reverse sequences
x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

# Pad sequences
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
def make_model(batch_size=None):
  source = Input(shape=(maxlen,), batch_size=batch_size, dtype=tf.int32, name='Input')
  embedding = Embedding(input_dim=max_features, output_dim=196, name='Embedding')(source)
  lstm = LSTM(32, name = 'LSTM')(embedding)
  predicted_var = Dense(1, activation='sigmoid', name='Output')(lstm)
  model = tf.keras.Model(inputs=[source], outputs=[predicted_var])
  model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='binary_crossentropy',
      metrics=['acc'])
  return model
   

In [ ]:
tf.keras.backend.clear_session()
training_model = make_model(batch_size = 128)
training_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (128, 500)                0         
_________________________________________________________________
Embedding (Embedding)        (128, 500, 196)           1960000   
_________________________________________________________________
LSTM (LSTM)                  (128, 32)                 29312     
_________________________________________________________________
Output (Dense)               (128, 1)                  33        
Total params: 1,989,345
Trainable params: 1,989,345
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import os
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.summary()

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.84.169.18:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 16607817266322147106)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2392198371098643890)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 12223693574578974246)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14938181096751591749)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 17355514275322943378)
INFO:tensorflow:*** Available Device: _Devi

In [ ]:
import time
start_time = time.time()

history = tpu_model.fit(x_train, y_train,
                    epochs=40,
                    batch_size=128 * 8,
                    validation_split=0.1)
print("--- %s seconds ---" % (time.time() - start_time))

tpu_model.save_weights('./tpu_model.h5', overwrite=True)

In [ ]:
inferencing_model = make_model(batch_size=None)
inferencing_model.load_weights('./tpu_model.h5')
inferencing_model.evaluate(x_test, y_test, batch_size=128 * 8)

25000/25000 [==============================] - 10s 400us/step


[1.010209317626953, 0.8240399995803833]

## Download the trained model weights to your local file system

In [ ]:
from google.colab import files

files.download('./tpu_model.h5')